# Non-Parametric-Learners

## kNN and Decision Trees

### Jonathan Torrez
#### January 30, 2017

## Lecture Objectives [morning-kNN]

* **Explain** one analogy demonstrating basic intuition of kNN algorithm
* **Describe** difference betwween parametric and non-parametric learning
* **Describe** pros and cons of parametric and non-parametric learning
* **Implement** pseudocode for kNN algorithm
* **Describe** effect of varying k (num_neigbors)
    * bias-variance tradeoff
* **Describe** pros and cons of kNN algorithm
* **Describe** curse of dimensionality
* **Explain** one analogy describing curse of dimensionality
* **State** common distance metrics used for kNN
* **Explain** importance of scaling for kNN
* **State** use case for kNN
* **Describe** variants of basic kNN algorithm

In [ ]:
#imports here

## Morning Exercise Objectives [morning-kNN]
Reinforces the following objectives

* **Implement** pseudocode for kNN algorithm
    * Writing a class that implements the kNN algorithm
* **State** common distance metrics used for kNN
    * Writing code that calculates two most common
* **Describe** effect of varying k (num_neigbors)
    * Cross validating to choose k
    * Plotting decision boundary can visualize the effect
* **Explain** importance of scaling for kNN
    * Plotting decision boundary can visualize the effect

## Objectives [afternoon-DT]

* **Explain** one analogy demonstrating intuition of decision tree algorithm
* **Describe** common measures for making splits in a decision tree
    * Classification
    * Regression
* **State** difference between how decision tree and regresssion tree make final predictions
* **Demonstrate** calculations to perform a split by hand, given data
* **Draw** graph of impurity function
* **Demonstrate** concept of recursion and **relate** it to decision trees
* **Implement** pseudocode for decision tree algorit
* **Analyze** prediction metrics for overfitting
* **Explain** methods for picking "best" tree
* **State** pruning techniques and pros/cons of each
    * Pre-pruning
    * Post-pruning
* **Demonstrate** use of cost penalty for post-pruning
* **Describe** pros/cons of decision tree algorithm

## Pair Exercise Objectives [afternoon-DT]
Reinforces the following objectives

* **Implement** pseudocode for decision tree algorithm
    * Writing code that implements CART algorithm
* **Explain** one analogy demonstrating intuition of decision tree algorithm
* **Describe** common measures for making splits in a decision tree
    * Classification
        * Writing code to calculate entropy and Gini
* **Demonstrate** concept of recursion and **relate** it to decision trees
    * * Writing code that implements CART algorithm
    
#### Extra Credit 1
* **State** pruning techniques and pros/cons of each
    * Pre-pruning
        * Writing code to implement pre-pruning
    * Post-pruning
        * Writing code to implement post-pruning
* **Demonstrate** use of cost penalty for post-pruning
        * Writing code to implement post-pruning
        
#### Extra Credit 2
* **Describe** common measures for making splits in a decision tree
    * Regression
        * Writing code to implement regression tree

## Distance Metrics

### Euclidean Distance

Straight line, L2

*Euclidean* distance is the distance metric you've probably heard of before:

$$ d(\mathbf{a}, \mathbf{b}) = ||\mathbf{a} - \mathbf{b}|| \ = \sqrt{\sum (a_i - b_i)^2} $$

### Cosine Similarity

Angle

*Cosine* similarity is another commonly used distance metric. It's measuring the angle between the two vectors.

$$ d(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}|| ||\mathbf{b}||} $$

### Manhattan Distance

City blocks, L1

$$\sum_i |a_i - b_i|$$

### Shannon Entropy

$$H(X) = E[I(X)] = E[log_2(\tfrac{1}{P(X)})]$$

$$= -E[log_2(P(X))]$$

$$H(X) = - \sum_i p_i log_2(p_i)$$


### Information Gain

$$\text{IG}(S, C) = H(S) - \sum_{C_i \in C} \frac{|C_i|}{|S|} H(C_i)$$


### Gini Index

$$\text{Gini}(S) = 1 - \sum_{i \in S} p_i^2$$

$$\text{IG}(S, C) = \text{Gini}(S) - \sum_{C_i \in C} \frac{|C_i|}{|S|} \text{Gini}(C_i)$$


### Weighted Voting

$$\frac{1}{d_i}$$

### k start

$$\sqrt{n}$$

### Hypercube stuff

$$ lim_{d \rightarrow \infty} \frac{V_{sphere}(R, d)}{V_{cube}(R, d)} = lim_{d \rightarrow \infty} \frac{\frac{\pi^{d/2}R^d}{\Gamma(d/2+1)}}{(2R)^d} = lim_{d \rightarrow \infty} \frac{\pi^{d/2}}{2^d\Gamma(d/2 + 1)} = 0 $$